In [8]:
from heapq import heappop, heappush
import matplotlib.pyplot as plt
import cv2
import numpy as np
from numpy import cos, pi
import soundfile as sf

In [2]:
# Código modificado desde: https://www.techiedelight.com/es/huffman-coding/

def isLeaf(root):
    return root.left is None and root.right is None

class Node:
    def __init__(self, ch, freq, left=None, right=None):
        self.ch = ch
        self.freq = freq
        self.left = left
        self.right = right
    # Anule la función `__lt__()` para hacer que la clase `Node` funcione con la cola de prioridad
    # tal que el elemento de mayor prioridad tiene la frecuencia más baja
    def __lt__(self, other):
        return self.freq < other.freq
 
 
# Atraviesa el árbol de Huffman y almacena los códigos de Huffman en un diccionario
def encode(root, s, huffman_code):
    if root is None:
        return
    if isLeaf(root):
        huffman_code[root.ch] = s if len(s) > 0 else 1
    encode(root.left, s + [0], huffman_code)
    encode(root.right, s + [1], huffman_code)
 
 
# Atraviesa el árbol de Huffman y decodifica la string codificada
def decode(root, index, s):
    if root is None:
        return index
    if isLeaf(root):
        print(root.ch, end=[])
        return index
    index = index + 1
    root = root.left if s[index] == [0] else root.right
    return decode(root, index, s)
 
 
# Construye Huffman Tree y decodifica el texto de entrada dado
def buildHuffmanTree(text, prt=0):
    if len(text) == 0:
        return

    freq = {i: text.count(i) for i in set(text)}
    pq = [Node(k, v) for k, v in freq.items()]
    heapq.heapify(pq)
 
    while len(pq) != 1:
        left = heappop(pq)
        right = heappop(pq)
        total = left.freq + right.freq
        heappush(pq, Node(None, total, left, right))

    root = pq[0]
    huffmanCode = {}
    encode(root, [], huffmanCode)

    if prt:
        print('Huffman Codes are:', huffmanCode)
        print('The original string is:', text)
        s = []
        for c in text:
            s += huffmanCode.get(c)
        print('The encoded string is:', s)
        print('The decoded string is:', end=' ')
        if isLeaf(root):
            while root.freq > 0:
                print(root.ch, end='')
                root.freq = root.freq - 1
        else:
            index = -1
            while index < len(s) - 1:
                index = decode(root, index, s)            
    return huffmanCode

In [3]:
def codificar(data, dic):
    s = []
    for c in data:
        s += dic.get(c)
    return s

In [4]:
def gen_01(data):
    if not isinstance(data, str):
        data = np.reshape(data, [1, -1])[0].tolist()
    dic = buildHuffmanTree(data)
    print(f'El diccionario tiene {len(dic)} elementos')
    arr = codificar(data, dic)
    print(f'La codificación de la data tiene {len(arr)} elementos')
    dic = list(dic.items())
    return arr, dic

In [5]:
def contrastComp(ch, a):
    new_ch = np.zeros(np.shape(ch), dtype=int)
    min= ch.min()
    max = ch.max()
    print('Para el receptor:',min, max)
    for i in range(ch.shape[0]):
        for j in range(ch.shape[1]):
            new_ch[i,j] = int(a*(ch[i,j]-min)/(max-min))
    return new_ch

In [11]:
def huff(contrast=False):
    I = cv2.imread("img_house.jpeg")
    cv2.imwrite('rgb_emisor.png', I)
    I = cv2.cvtColor(I, cv2.COLOR_RGB2YUV)
    Im = np.asarray(I)
    text = '¡Proyecto Numero 2, Principios de Comunicaciones Otoño 2022 EL4112! Modulación digital M-FSK'
    ch1, ch2, ch3 = cv2.split(Im)
    a = [150, 40, 35]
    if contrast:
        ch1 = contrastComp(ch1, a[0])
        ch2 = contrastComp(ch2, a[1])
        ch3 = contrastComp(ch3, a[2])
    cv2.imwrite('yuv_emisor.png', Im)
    data_cht = gen_01(text)
    data_ch1 = gen_01(ch1)
    data_ch2 = gen_01(ch2)
    data_ch3 = gen_01(ch3)
    path='C://Users//lesli//My Drive//La U//8vo semestre//EL4112 - Principios de Comunicación//Proyecto 2//'
    np.save(path+'dic_cht.npy', data_cht[1])
    np.save(path+'dic_ch1.npy', data_ch1[1])
    np.save(path+'dic_ch2.npy', data_ch2[1])
    np.save(path+'dic_ch3.npy', data_ch3[1])
    np.save(path+'data_cht.npy', data_cht[0])
    np.save(path+'data_ch1.npy', data_ch1[0])
    np.save(path+'data_ch2.npy', data_ch2[0])
    np.save(path+'data_ch3.npy', data_ch3[0])
    return data_cht[0], data_ch1[0], data_ch2[0], data_ch3[0]

In [12]:
data = huff(True)

Para el receptor: 10 251
Para el receptor: 99 167
Para el receptor: 104 188
El diccionario tiene 37 elementos
La codificación de la data tiene 439 elementos
El diccionario tiene 148 elementos
La codificación de la data tiene 285462 elementos
El diccionario tiene 41 elementos
La codificación de la data tiene 233826 elementos
El diccionario tiene 32 elementos
La codificación de la data tiene 203653 elementos


In [8]:
# longitud de cada codificacion de canal
largos = [len(data[0]), len(data[1]), len(data[2]), len(data[3])]

# frecuencia de muestreo (compu)
fs = 48000
Ts = 1/fs

# portadoras de canales de imagen
fmin=10000
fmax=20000
[fch1_0, fch2_0, fch3_0, fch1_1, fch2_1, fch3_1] = np.linspace(fmin, fmax, 6, dtype=int)
vec_f = [fch1_0, fch2_0, fch3_0, fch1_1, fch2_1, fch3_1] 

# portadoras de canal de texto
f0t = 3000
f1t = 4000

In [9]:
largos #Para el receptor

[439, 285462, 233826, 203653]

In [10]:
# periodos para cada simbolo
Np = 6
Nv = int(Np/(min(vec_f)*Ts))+1

NpT = 300
NvT = int(NpT/(f0t*Ts))+1

# Señal sincronizante

In [11]:
fsinc = [5000, 4000, 3000]
t_sinc = np.arange(0, 3, Ts)
sig_sinc = []
for f in range(3):
    sig_sinc[f*fs:(f+1)*fs] = np.cos(2*np.pi*fsinc[f]*t_sinc[f*fs:(f+1)*fs])
sig_sinc = np.array(sig_sinc)

# Señal de información

In [12]:
N = max(largos)
t_info = np.arange(0, N*Nv*Ts, Ts)

sig_t = np.zeros(t_info.shape[0])
sig_ch1 = np.zeros(t_info.shape[0])
sig_ch2 = np.zeros(t_info.shape[0])
sig_ch3 = np.zeros(t_info.shape[0])

# Modulación

In [13]:
def modulate(b, f0, f1, sig, i, Nv):
    for j in range(Nv):
        ti = Nv*i
        k = ti+j
        if b == 0:
            sig[k] = np.cos(2*np.pi*f0*t_info[ti+j])
        elif b == 1:
            sig[k] = np.cos(2*np.pi*f1*t_info[ti+j])
        else:
            print('ni uno')

In [14]:
for i in range(largos[0]):
    b = data[0][i]
    modulate(b, f0t, f1t, sig_t, i, NvT)

In [15]:
for i in range(largos[1]):
    b = data[1][i]
    modulate(b, fch1_0, fch1_1, sig_ch1, i, Nv)

In [16]:
for i in range(largos[2]):
    b = data[2][i]
    modulate(b, fch2_0, fch2_1, sig_ch2, i, Nv)

In [17]:
for i in range(largos[3]):
    b = data[3][i]
    modulate(b, fch3_0, fch3_1, sig_ch3, i, Nv)

# Unión señales y escritura .wav

In [18]:
sig_data= sig_t+sig_ch1+sig_ch2+sig_ch3
sig = np.append(sig_sinc, sig_data)
sf.write('mod.wav', sig, fs)